In [ ]:
import sklearn 
from sklearn import datasets
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
from sklearn import metrics
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.svm import LinearSVC,SVC
from scipy.stats import randint as randint
from scipy.stats import uniform
from sklearn.decomposition import TruncatedSVD
import pandas as pd
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.preprocessing import FunctionTransformer, MaxAbsScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score




categories = ["neg" , "pos"]

#read train files
docs_to_train = sklearn.datasets.load_files("C:/Users/nusai/Desktop/UNI/4th Year/Comp 551/P2/comp-551-imbd-sentiment-classification/train/train", 
      description=None, categories=categories,
      load_content=True, encoding='utf-8', shuffle=True, random_state=42)

#divide train set to train_data & cv_data
X_train, X_cv, y_train, y_cv = train_test_split(docs_to_train.data,
    docs_to_train.target, test_size=0.2)

In [ ]:
# reading test folder in order
def X_test():
    testdocs=[]
    i=0
    while i<25000:
      file1 = open("C:/Users/nusai/Desktop/UNI/4th Year/Comp 551/P2/comp-551-imbd-sentiment-classification/test/"+str(i)+".txt","r+",encoding="utf-8")
      l=file1.read()
      testdocs.append(l)
      i=i+1
      file1.close()
    return testdocs 

#creating a stopwords list 
def stopwords_list():
    
    words = [ ]
    with open('eng_st.txt','r') as f:
        for line in f:
            for word in line.split(','):
               words.append(word)
       
    return words

list_st = stopwords_list()

In [ ]:
# using a pipline model to extract features and preform features transformation
# then we slect one model to fit and predict our test_data             
chi_k = 5000
pclf = Pipeline([ 
    
    ('vect',CountVectorizer(ngram_range=(1,2),token_pattern=r'\b\w+\b', min_df=1)), #strip_accents='unicode')),#,stop_words= list_st)),ascii
    ('tfidf', TfidfTransformer()),
    #('dim_red', SelectKBest(chi2, chi_k)),
    #('scale', MaxAbsScaler()),
    ('norm', Normalizer()),
    #('SVD', TruncatedSVD(1000)),
    ('clf', LinearSVC(penalty ='l2',dual= True, C=5)),
    #('clf', KNeighborsClassifier(n_neighbors=25)),
    #('clf', SVC( gamma='auto', kernel='linear', C=7)),
    #('clf', LogisticRegression(C=5, class_weight='balanced')),#{"neg":1.2})),
    #('clf', DecisionTreeClassifier()),
])


pclf.fit(X_train, y_train)

y_pred = pclf.predict(X_cv)

#find f1 score of cv (fixed cv) 
print(metrics.classification_report(y_cv, y_pred, digits= 5,
    target_names=docs_to_train.target_names))

In [ ]:
# helper method to evalute a model
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
# using k-fold cross validation and report on the performance of the model variants
def validation_pipeline():          
    params = {"vect__ngram_range": [(1,2),(1,3)],  
         "tfidf__use_idf": [True,False],
         #SVC param
         "clf__C": np.arange(4, 9, 0.5),
         }
    
    seed = 551 # Very important for repeatibility in experiments!
    
    random_search = RandomizedSearchCV(pclf, param_distributions = params, cv=2, verbose = 10, random_state = seed, n_iter = 5)
    random_search.fit(X_train, y_train)
    
    report(random_search.cv_results_)
    y_pred = random_search.predict(X_cv)
    
    print(metrics.classification_report(y_cv, y_pred,digits= 5,
        target_names=docs_to_train.target_names))
       
#validation_pipeline()   
 
    
#other method to calculate the score on k-fold cv 
# only one cv_scores() or validation_pipline should be called    
def cv_scores():
    
   scores = cross_val_score(pclf, docs_to_train.data, docs_to_train.target, cv=3, verbose = 10, scoring='f1_macro')
   print(scores)
   print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))    


#cv_scores()
         

In [ ]:
#helper method to create csv file    
def create_csv(): 
    
    testdocs = X_test()
    y_pred_new = pclf.predict(testdocs)    
    df = pd.DataFrame(columns=['Id' , 'Category'])
    
    for i in range(len(y_pred_new)): 
        #csv.write(i,y_pred_new[i])
    	#print("X=%s, Predicted=%s" % (i, y_pred_new[i]))
        list = [i,y_pred_new[i]]
        df.loc[i]=list
        
    print(df)
    df.to_csv('ML_p2.csv')
    
create_csv()  
    
    
   